# Synthèse de textures par réseau convolutif (filtres aléatoires)

## Mise en Place

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import numpy as np

from PIL import Image
import matplotlib.pyplot as plt
from io import BytesIO
from urllib.request import urlopen
import os

from utils import prep_img, denormalize, to_pil, randomize_layer_, randomize_model_, register_model_hooks, unregister_model_hooks

In [ ]:
# fetch images
texture_imgnames = ["bois.png", "briques.png", "mur.png",
                    "tissu.png", "nuages.png", "pebbles.jpg", "wall1003.png"]
#TODO use urllib instead 
#import wget
for fname in texture_imgnames:
    os.system("wget -c https://www.idpoisson.fr/galerne/mva/" + fname)

In [ ]:
# device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device is", device)

## Fonctions de perte

In [ ]:
def gramm(tnsr: torch.Tensor) -> torch.Tensor:
    """Computes Gram matrix for the input batch tensor.
    Args: tnsr (torch.Tensor): input tensor of the Size([B, C, H, W]).
    Returns:  G (torch.Tensor): output tensor of the Size([B, C, C]).
    """
    b, c, h, w = tnsr.size()
    F = tnsr.view(b, c, h * w)
    G = torch.bmm(F, F.transpose(1, 2))
    G.div_(h * w)
    return G


def gram_loss(input: torch.Tensor, gramm_target: torch.Tensor, weight: float = 1.0):
    """Computes the MSE loss between the Gram matrix of the input and the target
    Gtam matrix. 
    """
    loss = weight * F.mse_loss(gramm(input), gramm_target)
    return loss

#TODO define content_loss, texture_loss

In [ ]:
# layers to use in the texture synthesis:
TARGET_LAYERS = [1, 6, 11, 20, 29]  # 1rst Conv2d and all MaxPool2d (after ReLU)

In [ ]:
cnn = models.vgg19(pretrained=False).features.to(device).eval()

In [ ]:
def build_ranvgg_(cnn):
    # remplacement des poids du modèle par un bruit blanc:
    randomize_model_(cnn)
    cnn.requires_grad_(False)  # on empêche le modèle d'apprendre
    layers_to_build = [
        idx for idx, layer in enumerate(cnn) if isinstance(layer, nn.Conv2d)
    ]
    # on enregistre la sortie des couches qu'on construit
    outputs, handles = register_model_hooks(cnn, layers_to_build)
    # TODO do stuff here
    unregister_model_hooks(handles)  # on enlève les handles du modèle
    return cnn


In [ ]:
build_ranvgg_(cnn)